In [ ]:
import concurrent.futures

def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()

def write_file(data, filename):
    with open(filename, 'w') as f:
        f.write(data)

files = ["file1.txt", "file2.txt", "file3.txt"]

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    contents = list(executor.map(read_file, files))

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(write_file, contents, ["output1.txt", "output2.txt", "output3.txt"])